# Logging
To keep track of what is happening in your code, you can use the provided logging module.
Below you can see an example of how to use it. The `enable_file_logger` utility function will
create a file logger that will write to the file `TutorialLogging.log` in the `tutorial_output` directory.

In [1]:
# logging settings
from qsprpred.logs.utils import enable_file_logger
logSettings = enable_file_logger(
    log_folder = '../../tutorial_output/',
    filename = 'TutorialLogging.log',
    debug = False,
    disable_existing_loggers = False
)

A utility function is provided to save your conda environment to a file. This is useful if you want to share your code with others, or if you want to recreate your environment at a later time.

In [2]:
from qsprpred.logs.utils import export_conda_environment

# save the environment to a yaml file
export_conda_environment("../../tutorial_output/TutorialLoggingEnvironment.yml")

Environment exported to ../../tutorial_output/TutorialLoggingEnvironment.yml successfully!


Below we run a standard data preparation and modelling workflow to show logging output.

In [3]:
from qsprpred.data import RandomSplit
from qsprpred.data.descriptors.sets import FingerprintSet
from qsprpred.data.descriptors.calculators import MoleculeDescriptorsCalculator
from qsprpred.data import QSPRDataset
# Standard data preparation
import os

os.makedirs("../../tutorial_output/data", exist_ok=True)

dataset = QSPRDataset.fromTableFile(
  	filename='../../tutorial_data/A2A_LIGANDS.tsv', 
  	store_dir="../../tutorial_output/data",
  	name="LoggingTutorialDataset",
  	target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
  	random_state=42
)

feature_calculator = MoleculeDescriptorsCalculator(
    desc_sets = [FingerprintSet(fingerprint_type="MorganFP", radius=3, nBits=2048)]
)

dataset.prepareDataset(
    split=RandomSplit(test_fraction=0.2, dataset=dataset),
    feature_calculators=[feature_calculator],
)

dataset.save()

In [4]:
# Standard model training
from qsprpred.models import SklearnModel
from sklearn.neighbors import KNeighborsRegressor
from qsprpred.models import CrossValAssessor, TestSetAssessor

os.makedirs("../../tutorial_output/models", exist_ok=True)

model = SklearnModel(
    base_dir = '../../tutorial_output/models',
    data = dataset,
    alg = KNeighborsRegressor,
    name = 'TutorialLoggingModel'
)

CrossValAssessor('r2')(model)
TestSetAssessor('r2')(model)

model.fitAttached()

model.save()

/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/helle/.conda/envs/qsprpred/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed

'/zfsdata/data/helle/01_MainProjects/03_QSPRPred/Scripts/QSPRpred/tutorials/tutorial_output/models/TutorialLoggingModel/TutorialLoggingModel_meta.json'